**CPLEX installation guide:** Add following line to your `.bash_profile` (for macOS) after installing IBM ILOG CPLEX Optimization Studio locally using installer from IBM. (note the path will change if you are using different python or OS)

```export PYTHONPATH="/Applications/CPLEX_Studio1210/cplex/python/3.6/x86-64_osx"```

In [1]:
import cplex
print(cplex.__path__)  # make sure the path is the one we pick above

['/Applications/CPLEX_Studio1210/cplex/python/3.6/x86-64_osx/cplex']


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from util_io import (
    init, finalize, dump_conf, assigned_day_to_family_on_day, assigned_day_to_occupancy
)
from util_cost import (
    cal_total, n_people, family_id_choice_to_pref_cost, cal_total_preference, cal_total_accounting,
    nd_ndp1_to_account_penality, family_id_days_to_pref_cost
)
from util_cost import choices as family_pref
from util_check import deep_check, check_valid_all

In [3]:
from docplex.mp.model import Model

## Parameters

In [4]:
# constants #
N_families = 5000
N_days = 100
N_min_people = 125
N_max_people = 300
# constants #

# params #
path_init_conf =     '../output/m24-improved-2.csv'
path_dump_improved = '../output/m24-improved-random.csv' # output solution

num_cpu_cores = 6
#time_limit = -1 # unlimited
time_limit = 10*60  # in s

select_days = 75  # N of worst pref cost days selected
window_run = 30*12*5*12
occupancy_diff = 0  # +- the occupancy of input solution for each day
occupancy_diff_low = 0  # +- the occupancy of input solution for each day
max_family_rank = 6  # maximum number of rank of the preference days for each family
use_hint = True      # use current input as hint
occupancy_count_as_variables = False  # use occupancy_counts as variable (seem faster)
redundant_occupancy_constraints = True  # use redundant constraints
min_choice_0_families = 3742   # minimum number of families that are at their choice 0
target_pref_cost = 0 # 62868
target_pref_cost_error = 0
target_pref_cost_lower = 0
target_accounting_cost = 0 # 6020.043432
target_accounting_cost_error = 0
target_accounting_cost_lower = 0
max_accounting_cost_per_day = 600

In [5]:
N_families - 62868 / 50

3742.6400000000003

In [6]:
families = range(N_families)
days = range(1, N_days + 1)
allowed_occupancy = range(N_min_people, N_max_people + 1)
possible_family_sizes = np.unique(n_people)

In [7]:
# occupancy pairs [o, o_next] limited by accounting cost
viable_nd_ndp1 = nd_ndp1_to_account_penality <= max_accounting_cost_per_day

In [8]:
# Possible choice for the family
# last choice is any day that is not on the family's preferred days
N_choices_ori = family_id_choice_to_pref_cost.shape[1]
N_choices = min(N_choices_ori, max_family_rank)
print('Limit family choice rank:', N_choices_ori, '->', N_choices)

Limit family choice rank: 11 -> 6


In [9]:
N_family_pref = min(N_choices, N_choices_ori - 1)
print(N_family_pref)

6


In [10]:
# day to dictionary of families who choose this day with value as preference rank
days_family_prefered = [{} for day in range(N_days+1)]  # day = 0 should not be used
for family, pref in enumerate(family_pref):
    for rank, day in enumerate(pref):
        if rank < N_family_pref:
            days_family_prefered[day][family] = rank

## MIP progress listener (need initial score)

In [11]:
from docplex.mp.progress import TextProgressListener
from docplex.mp.progress import ProgressClock
from docplex.mp.progress import SolutionRecorder

See http://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.progress.html#docplex.mp.progress.ProgressClock for progress clock parameters meaning

In [12]:
class MyProgressListener(SolutionRecorder):
    def __init__(self, initial_score=999999, clock=ProgressClock.Gap, absdiff=None, reldiff=None):
        super(MyProgressListener, self).__init__(clock, absdiff, reldiff)
        self.current_objective = initial_score
        
    def notify_solution(self, sol):
        if self.current_progress_data.current_objective >= self.current_objective:
            return
        print ('Improved solution')
        super(MyProgressListener, self).notify_solution(sol)
        self.current_objective = self.current_progress_data.current_objective
        assigned_day_new_raw = np.ones(N_families, dtype='int32') * -1
        for family, choice in sol.get_value_dict(assignment_matrix, keep_zeros=False):
            assigned_day_new_raw[family] = family_pref[family, choice] if choice < N_family_pref else -1
        solution = pd.DataFrame(data=families, columns = ['family_id'])
        solution['assigned_day'] = assigned_day_new_raw
        #score = cost_function(preds)
        #print('Score: ' + str(score))        
        solution.to_csv(path_dump_improved, index=False)
        
    def get_solutions(self):
        return self._solutions

## other function 

In [13]:
def distribute_unpreferred_day(assigned_day, unpreferred_day_counts_sol, n_people):
    """ Distribute unpreferred day to each family who has -1 day assigned """
    assigned_day = assigned_day.copy()
    unpreferred_days = {size: [] for size in possible_family_sizes}
    for size in possible_family_sizes:
        for day, quota in enumerate(unpreferred_day_counts_sol[size]):
            unpreferred_days[size] = unpreferred_days[size] + [day] * quota
    unpreferred_day_headers = {size: 0 for size in possible_family_sizes}
    for family, (day, size) in enumerate(zip(assigned_day, n_people)):
        if day == -1:
            assigned_day[family] = unpreferred_days[size][unpreferred_day_headers[size]]
            unpreferred_day_headers[size] += 1
    return assigned_day

## Load initial

In [14]:
assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
print('Init config:')
try:
    is_valid = deep_check(assigned_day, family_on_day, occupancy)
except:
    is_valid = False
initial_score = cal_total(assigned_day, occupancy)
print('Valid solution: ', is_valid)
print('Total score:    ', initial_score)
print('Preference cost:', cal_total_preference(assigned_day))
print('Accounting cost:', cal_total_accounting(occupancy))

# init_occupancy_counts = {o: 0 for o in allowed_occupancy}
# for o in occupancy[1:-1]:
#     init_occupancy_counts[o] += 1
# init_occupancy_counts = pd.Series(init_occupancy_counts)

# accounting_cost_per_day = pd.Series({
#     day: nd_ndp1_to_account_penality[occupancy[day], occupancy[day+1]].astype('float32')
#     for day in days
# })

### Day Family Choice rank

# assigned_day_pref_rank = (family_pref == assigned_day.reshape(-1, 1)).argmax(axis=1)

# pd.Series(assigned_day_pref_rank).value_counts()

# df_assigned_day = pd.DataFrame({'day': assigned_day, 'rank': assigned_day_pref_rank, 'n': n_people})

# df_day_pref_rank = df_assigned_day.groupby(['day', 'rank'])[['n']].sum().reset_index().pivot(index='day', columns='rank', values='n').fillna(0).astype('int32')

Read initial configs...
Read config completed.
Init config:
deep check: everything looks fine.
Valid solution:  True
Total score:     68924.84720657553
Preference cost: 63308
Accounting cost: 5616.847206575537


In [15]:
best_score = initial_score

## Loop

In [16]:
from itertools import product

In [17]:
%%time
days_qu = []
#for i, p in product(range(window_run), range(int(N_days / select_days)+1)):
for i in range(window_run):
    if i%10 == 0:
        assigned_day_o, family_on_day_o, occupancy_o = init(path_conf=path_init_conf)
        print('Init config:')
        try:
            is_valid = deep_check(assigned_day_o, family_on_day_o, occupancy_o)
        except:
            is_valid = False
        new_initial_score = cal_total(assigned_day_o, occupancy_o)
        if new_initial_score < best_score and is_valid:
            print('Total score:    ', new_initial_score)
            print('Preference cost:', cal_total_preference(assigned_day_o))
            print('Accounting cost:', cal_total_accounting(occupancy_o))
            print('using new conf')
            best_score = new_initial_score
            assigned_day = assigned_day_o
            family_on_day = family_on_day_o
            occupancy = occupancy_o
    
    families_cost = family_id_days_to_pref_cost[np.arange(N_families), assigned_day]
    df_family = pd.DataFrame({'day': assigned_day, 'cost': families_cost})
    day_pref_cost = df_family.groupby('day')[['cost']].sum()
#     iw = int(i / select_days)
#     expensive_days = day_pref_cost.sort_values('cost', ascending=False).iloc[
#         i + p * (select_days + iw) : i + (p+1) * (select_days + iw)].index.values
    iw = int(2 * i / window_run)
    expensive_days = list(np.random.choice(days, select_days+iw - len(days_qu), replace=False))
    print('[', i, ']', expensive_days+days_qu)
    #print('[', i, ',', p,']', expensive_days)
    if len(expensive_days) == 0:
            continue
    
    # limit the occupancy choice to +- occupancy_diff of current solution
    search_occupancy = {}
    for day in days:
        if day in expensive_days:
            search_occupancy[day] = range(N_min_people, N_max_people+1)
        elif occupancy[day] == N_min_people:
            search_occupancy[day] = range(N_min_people, occupancy[day] + occupancy_diff_low + 1)
        else:
            search_occupancy[day] = range(max(occupancy[day] - occupancy_diff, N_min_people),
                                          min(occupancy[day] + occupancy_diff, N_max_people) + 1)

    # for i, x in search_occupancy.items():
    #     print(i, x)

    ## DOCplex model

    solver = Model('')

    if num_cpu_cores > 0:
        solver.context.cplex_parameters.threads = num_cpu_cores
        #print('Set num threads:', num_cpu_cores)
    print('Num treads:', solver.context.cplex_parameters.threads)
    if time_limit > 0:
        solver.set_time_limit(time_limit)

    solver.parameters.mip.tolerances.mipgap = 0  # set mip gap to 0

    ## Variables

    # Variables
    # assignment matrix[family, pref_rank]
    assignment_matrix = solver.binary_var_matrix(families, range(N_choices), 'x')

    # unpreferred_day_counts[day, size]
    if N_choices_ori <= N_choices:
        print('using unpreferred day counts')
        ub = int(N_max_people / possible_family_sizes.min())
        unpreferred_day_counts = solver.integer_var_matrix(days, possible_family_sizes, lb=0, ub=ub, name='d')
        print(len(unpreferred_day_counts))    

    # Occupancy matrix [day, N_d, N_d+1]
    occupancy_keys_list = []
    for day in days:
        if day < N_days:
            for o in search_occupancy[day]:
                for o_next in search_occupancy[day + 1]:
                    if viable_nd_ndp1[o, o_next]:
                        occupancy_keys_list.append((day, o, o_next))
        else:
            # last day
            for o in search_occupancy[day]:
                if viable_nd_ndp1[o, o]:
                    occupancy_keys_list.append((day, o))
    occupancy_matrix = solver.binary_var_dict(occupancy_keys_list, name='o')

    ## Constraints

    ### constraint 1: each family only take one day (choice)

    # Constraints
    # constraint 1: each family only take one day (choice)
    solver.add_constraints_([
        solver.sum([assignment_matrix[family, c] for c in range(N_choices)]) == 1 
        for family in families
    ])

    ### constraint: choices limit

    if min_choice_0_families > 0:
        solver.add_constraint_(
            solver.sum([assignment_matrix[family, 0] for family in families]) >= min_choice_0_families
        )

    ### occupancy counts

    # constraint 2: each day can only have 125-300 people

    # occupancy count [intermediate variables]

    if occupancy_count_as_variables:
        lbs = [min(search_occupancy[day]) for day in days]
        ubs = [max(search_occupancy[day]) for day in days]
        occupancy_counts = solver.integer_var_dict(days, lb=lbs, ub=ubs, name='oc')

        for day in days:
            # find those family who like this day
            family_prefered = days_family_prefered[day]
            solver.add_constraint_(
                occupancy_counts[day] == (
                    solver.sum(
                        [assignment_matrix[family, pref_rank] * n_people[family] 
                         for family, pref_rank in family_prefered.items()]
                    ) + (
                        solver.sum(
                            [unpreferred_day_counts[day, size] * size for size in possible_family_sizes]
                        ) if N_choices >= N_choices_ori else 0
                    )
                )
            )
    else:
        occupancy_counts = {}
        for day in days:
            # find those family who like this day
            family_prefered = days_family_prefered[day]
            occupancy_counts[day] = (
                solver.sum(
                    [assignment_matrix[family, pref_rank] * n_people[family] 
                     for family, pref_rank in family_prefered.items()]
                ) + (
                    solver.sum(
                        [unpreferred_day_counts[day, size] * size for size in possible_family_sizes]
                    ) if N_choices >= N_choices_ori else 0
                )
            )

    if not occupancy_count_as_variables:
        for day in days:
            solver.add_range(min(search_occupancy[day]), 
                             occupancy_counts[day], 
                             max(search_occupancy[day]))

    ### constraint 3: unpreferred day family count conservation for each family size

    # constraint 3: unpreferred day family count conservation for each family size

    family_size_to_family_ids = {
        size: np.where(n_people == size)[0] for size in possible_family_sizes
    }

    if N_choices >= N_choices_ori:
        solver.add_constraints_([
            solver.sum([assignment_matrix[family, N_choices - 1]
                        for family in family_size_to_family_ids[size]])
            == solver.sum([unpreferred_day_counts[day, size] for day in days])
            for size in possible_family_sizes
        ])

    ### Occupancy boolean matrix normalization

    # occupancy boolean matrix normalization
    # each day only take 1 occupancy value
    for day in days:
        if day < N_days:
            occupancy_normalization = solver.sum([
                occupancy_matrix[day, o, o_next] 
                for o in search_occupancy[day]
                for o_next in search_occupancy[day + 1]
                if viable_nd_ndp1[o, o_next]
            ])
        else:
            occupancy_normalization = solver.sum([
                occupancy_matrix[day, o] 
                for o in search_occupancy[day]
                if viable_nd_ndp1[o, o]
            ])
        solver.add_constraint_(occupancy_normalization == 1)

    ### constrain 4: link occupancy boolean matrix to occupancy count

    for day in days:
        if day < N_days:
            sum_from_occupancy_matrix = solver.sum([
                occupancy_matrix[day, o, o_next] * o 
                for o in search_occupancy[day]
                for o_next in search_occupancy[day + 1]
                if viable_nd_ndp1[o, o_next]
            ])
        else:
            sum_from_occupancy_matrix = solver.sum([
                occupancy_matrix[day, o] * o 
                for o in search_occupancy[day]
                if viable_nd_ndp1[o, o]            
            ])
        solver.add_constraint_(occupancy_counts[day] == sum_from_occupancy_matrix)

    # next day occupancy consistency
    solver.add_constraints_([
        occupancy_counts[day + 1] == solver.sum([
            occupancy_matrix[day, o, o_next] * o_next 
            for o in search_occupancy[day]
            for o_next in search_occupancy[day + 1]
            if viable_nd_ndp1[o, o_next]            
        ])
        for day in days if day < N_days
    ])

    # redudant constraints
    if redundant_occupancy_constraints:
        for day in days:
            if day + 1 < N_days:
                solver.add_constraints_([
                    solver.sum([
                        occupancy_matrix[day, o_other, o] 
                        for o_other in search_occupancy[day] if viable_nd_ndp1[o_other, o]
                    ]) == solver.sum([
                        occupancy_matrix[day + 1, o, o_other]
                        for o_other in search_occupancy[day + 2] if viable_nd_ndp1[o, o_other]
                    ])
                    for o in search_occupancy[day + 1]
                ])
        solver.add_constraints_([
            solver.sum([
                occupancy_matrix[N_days - 1, o_other, o] 
                for o_other in search_occupancy[N_days - 1] if viable_nd_ndp1[o_other, o]
            ]) == occupancy_matrix[N_days, o] if viable_nd_ndp1[o, o] else 0
            for o in search_occupancy[N_days]
        ])

    ### Preference cost

    family_pref_cost = solver.sum([
        assignment_matrix[family, c] * family_id_choice_to_pref_cost[family, c]
        for family in families for c in range(1, N_choices)
    ])

    if target_pref_cost > 0:
        if target_pref_cost_error > 0:
            print('Limit preference cost in range')
            solver.add_range(
                target_pref_cost - target_pref_cost_error,
                family_pref_cost,
                target_pref_cost + target_pref_cost_error
            )
        else:
            print('Limit preference cost exactly')
            solver.add_constraint_(family_pref_cost == target_pref_cost)
    elif target_pref_cost_lower > 0:
        print('Lower bound preference cost')
        solver.add_constraint_(family_pref_cost >= target_pref_cost_lower)

    ### Accounting cost

    accounting_cost = (
        solver.sum([
            occupancy_matrix[day, o, o_next] * nd_ndp1_to_account_penality[o, o_next]
            for day in days if day < N_days
            for o in search_occupancy[day] for o_next in search_occupancy[day + 1]
            if viable_nd_ndp1[o, o_next] and o > N_min_people
        ]) +
        solver.sum([
            occupancy_matrix[N_days, o] * nd_ndp1_to_account_penality[o, o]
            for o in search_occupancy[N_days]
            if viable_nd_ndp1[o, o] and o > N_min_people  
        ])
    )

    if target_accounting_cost > 0:
        if target_accounting_cost_error > 0:
            print('Range limit accounting cost')
            solver.add_range(
                target_accounting_cost - target_accounting_cost_error,
                accounting_cost,
                target_accounting_cost - target_accounting_cost_error
            )
    elif target_accounting_cost_lower > 0:
        print('Lower bound accounting cost')
        solver.add_constraint_(accounting_cost >= target_accounting_cost_lower)

    # ==== Objective ====
    solver.minimize(family_pref_cost + accounting_cost)  # family_pref_cost + 

    ## Hint

    if use_hint:
        print('Using hint!')

        from docplex.mp.solution import SolveSolution
        var_value_map = {}

        for family in families:
            for c in range(N_choices):
                var_value_map[assignment_matrix[family, c]] = float(
                    assigned_day[family] == family_pref[family, c]
                )
        for day in days:
            if day < N_days:
                for o in search_occupancy[day]:
                    for o_next in search_occupancy[day + 1]:
                        if viable_nd_ndp1[o, o_next]:
                            var_value_map[occupancy_matrix[day, o, o_next]] = float(
                                (occupancy[day] == o) and (occupancy[day + 1] == o_next)
                            )
                        else:
                            assert not ((occupancy[day] == o) and (occupancy[day + 1] == o_next)), \
                            'Hint not valid at (%i, %i, %i)'%(day, o, o_next)
        for o in search_occupancy[N_days]:
            if viable_nd_ndp1[o, o]:
                var_value_map[occupancy_matrix[N_days, o]] = float(occupancy[N_days] == o)
            else:
                assert not (occupancy[N_days] == o), \
                'Hint not valid at (%i, %i, %i)'%(N_days, o, o)

        if occupancy_count_as_variables:
            for day in days:
                var_value_map[occupancy_counts[day]] = float(occupancy[day])

        init_solution = SolveSolution(solver, var_value_map)
        solver.add_mip_start(init_solution)

    ## Solve

    # print progress
    my_progress_listener = MyProgressListener(initial_score=best_score, clock=ProgressClock.Objective)
    solver.add_progress_listener(
    #     TextProgressListener(clock=ProgressClock.Gap)
        my_progress_listener
    )  


    print('N of variables (binary, int):', solver.number_of_variables, 
          '(', solver.number_of_binary_variables, ',', solver.number_of_integer_variables, ')')
    print('N of constraints:', solver.number_of_constraints)
    print('Time limit:', solver.get_time_limit())

    # Solve
    sol = solver.solve(log_output=True)

    if sol is None:
        sol = my_progress_listener.get_solutions()[-1]

    print('Solution status:', solver.get_solve_status())
    print('Total cost:', sol.objective_value, sol.get_objective_value())
    print("Time:", '%.3f' % solver.get_solve_details().time, "s")

    ## Get Solution

    assigned_day_new_raw = np.ones(assigned_day.shape, dtype='int32') * -1
    for family, choice in sol.get_value_dict(assignment_matrix, keep_zeros=False):
        assigned_day_new_raw[family] = family_pref[family, choice] if choice < N_family_pref else -1

    if N_choices >= N_choices_ori:
        unpreferred_day_counts_sol_dict = sol.get_value_dict(unpreferred_day_counts)
        unpreferred_day_counts_sol = {
            size: [0]+[int(unpreferred_day_counts_sol_dict[day, size]) for day in days]
            for size in possible_family_sizes
        }

        print('Unpreferred families slots:')
        {size: sum(counts) for size, counts in unpreferred_day_counts_sol.items()}

    if N_choices >= N_choices_ori:
        assigned_day_new = distribute_unpreferred_day(assigned_day_new_raw, unpreferred_day_counts_sol, n_people)
    else:
        assigned_day_new = assigned_day_new_raw

    print('N family unpreferred assigned:', (~(assigned_day_new == assigned_day_new_raw)).sum())

    family_on_day_new = assigned_day_to_family_on_day(assigned_day_new)
    occupancy_new = assigned_day_to_occupancy(assigned_day_new)

    try:
        is_valid = deep_check(assigned_day_new, family_on_day_new, occupancy_new)
    except:
        is_valid = False
    new_score = cal_total(assigned_day_new, occupancy_new)
    print('Valid solution: ', is_valid)
    print('Total score:    ', new_score, '(', new_score - best_score, ')')
    print('Preference cost:', cal_total_preference(assigned_day_new))
    print('Accounting cost:', cal_total_accounting(occupancy_new))
    
    # end the model
    solver.end()
    
    # update
    if new_score < best_score:
        occupancy_change = (occupancy_new != occupancy)[1:-1]
        days_qu = list(np.array(days)[occupancy_change])  # search them first
        
        dump_conf(assigned_day_new, path_dump_improved)
        
        best_score = new_score
        assigned_day = assigned_day_new
        family_on_day = family_on_day_new
        occupancy = occupancy_new

Read initial configs...
Read config completed.
Init config:
deep check: everything looks fine.
[ 0 ] [3, 97, 31, 74, 89, 39, 82, 87, 43, 25, 92, 45, 16, 49, 9, 42, 1, 51, 50, 66, 23, 53, 81, 19, 78, 20, 29, 36, 52, 77, 33, 84, 6, 12, 18, 70, 91, 57, 28, 40, 35, 58, 100, 47, 5, 24, 62, 44, 69, 48, 60, 37, 10, 4, 76, 80, 13, 32, 88, 65, 63, 79, 71, 21, 8, 59, 98, 27, 2, 99, 75, 46, 72, 54, 34]
Num treads: threads:int(6)
Using hint!
N of variables (binary, int): 904824 ( 904824 , 0 )
N of constraints: 18449
Time limit: 300.0
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 6
CPXPARAM_RandomSeed                              201903125
CPXPARAM_TimeLimit                               300
CPXPARAM_MIP_Tolerances_MIPGap                   0
Presolve time = 0.22 sec. (131.79 ticks)
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 68924.8472.
Tried ag

   2052  1615    68913.7545   444    68924.8472    68907.9339   202487    0.02%
   2097  1708    68916.2716   325    68924.8472    68907.9339   214416    0.02%
   2202  1735    68916.6883   374    68924.8472    68908.0085   222793    0.02%
   2260  1840    68911.9611   447    68924.8472    68908.1174   235798    0.02%
   2336  1836    68920.5060   376    68924.8472    68908.2792   235733    0.02%
   2437  1933    68917.9468   405    68924.8472    68908.2887   247750    0.02%
   2502  2035    68918.8274   499    68924.8472    68908.3275   263363    0.02%
   2608  2156    68924.1898   421    68924.8472    68908.3275   277274    0.02%
   2684  2173    68916.1221   373    68924.8472    68908.4687   279779    0.02%
Elapsed time = 140.22 sec. (154295.60 ticks, tree = 115.06 MB, solutions = 1)
   2739  2223    68923.7541   402    68924.8472    68908.4687   286808    0.02%
   2829  2287    68914.9244   362    68924.8472    68908.5649   295878    0.02%
   2903  2312    68918.4680   315    68924

      0     0    68830.2388   449    68924.8472     Cuts: 268     8439    0.14%
      0     0    68847.8509   449    68924.8472     Cuts: 213    10095    0.11%

Repeating presolve.
Tried aggregator 2 times.
MIP Presolve eliminated 1310 rows and 52550 columns.
MIP Presolve modified 265 coefficients.
Aggregator did 1564 substitutions.
Reduced MIP has 5706 rows, 78834 columns, and 385922 nonzeros.
Reduced MIP has 78779 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.50 sec. (447.23 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 5706 rows, 78834 columns, and 385922 nonzeros.
Reduced MIP has 78779 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.24 sec. (215.38 ticks)
Represolve time = 1.49 sec. (1461.74 ticks)
Probing time = 0.06 sec. (50.27 ticks)
Clique table members: 26194.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 6 threads.
Root relaxation so

Reduced MIP has 14935 rows, 683227 columns, and 3369060 nonzeros.
Reduced MIP has 683154 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 3.01 sec. (2459.19 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 14935 rows, 683227 columns, and 3369060 nonzeros.
Reduced MIP has 683154 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 2.57 sec. (1766.46 ticks)
Probing time = 0.69 sec. (404.00 ticks)
Clique table members: 67392.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 6 threads.
Root relaxation solution time = 16.12 sec. (18602.40 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                        68924.8472        0.0000           100.00%
      0     0    68244.5743   210    68924.8472    68244.5743      102    0.99%
      0     0    68648.99

    452   258    68918.7980   246    68924.8472    68844.1037   112733    0.12%
    454   273    68920.0275   228    68924.8472    68844.1037   114577    0.12%
    457   276    68922.0969   174    68924.8472    68844.1037   114738    0.12%
    462   281    68923.0386   217    68924.8472    68844.1037   115071    0.12%
    469   216    68856.5195   684    68924.8472    68847.0044    95220    0.11%
    471   272    68855.5948   744    68924.8472    68847.2482   117748    0.11%
    473   284    68913.7798   452    68924.8472    68847.2482   118369    0.11%
Elapsed time = 275.24 sec. (271728.38 ticks, tree = 20.81 MB, solutions = 1)
    475   262    68914.1192   452    68924.8472    68847.7724   116327    0.11%
    487   293    68920.7122   507    68924.8472    68847.7724   123353    0.11%
    508   298    68922.0028   509    68924.8472    68847.7724   125906    0.11%
    524   291    68921.6694   453    68924.8472    68849.2296   130083    0.11%
    533   295    68921.6180   408    68924.

...

Parallel mode: deterministic, using up to 6 threads.
Root relaxation solution time = 7.84 sec. (7797.17 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                        68924.8472    68557.7731             0.53%
      0     0    68557.7731   467    68924.8472    68557.7731     7050    0.53%
      0     0    68656.2656   467    68924.8472     Cuts: 307     9236    0.39%
      0     0    68690.5089   467    68924.8472     Cuts: 240    10656    0.34%
Detecting symmetries...

Repeating presolve.
Tried aggregator 2 times.
MIP Presolve eliminated 993 rows and 93529 columns.
MIP Presolve modified 323 coefficients.
Aggregator did 69 substitutions.
Reduced MIP has 10544 rows, 206457 columns, and 998284 nonzeros.
Reduced MIP has 206384 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.11 sec. (802.90 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduce

   2227  1082    68924.5833   494    68924.8472    68920.5026   377980    0.01%
   2264  1078    68923.9865   492    68924.8472    68920.5699   376628    0.01%
   2297  1088    68924.4869   491    68924.8472    68920.6437   387246    0.01%
   2332  1087    68922.0913   447    68924.8472    68920.6950   388665    0.01%
   2368  1103        cutoff          68924.8472    68920.7431   407592    0.01%
   2412  1105    68924.1188   424    68924.8472    68920.7487   412065    0.01%
   2459  1109    68924.3012   303    68924.8472    68920.8273   417254    0.01%
   2511  1109    68923.0776   448    68924.8472    68920.9007   420178    0.01%
   2561  1122    68924.7389   518    68924.8472    68920.9851   433128    0.01%
   2605  1123        cutoff          68924.8472    68920.9851   447330    0.01%
Elapsed time = 176.14 sec. (189997.42 ticks, tree = 75.01 MB, solutions = 1)
   2662  1143    68922.2608   484    68924.8472    68921.0932   457868    0.01%
   2721  1133    68924.0418   430    68924.

Clique table members: 21965.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 6 threads.
Root relaxation solution time = 2.90 sec. (2891.30 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                        68924.8472    68759.8970             0.24%
      0     0    68759.8970   423    68924.8472    68759.8970     7255    0.24%
      0     0    68805.1216   423    68924.8472     Cuts: 275     9312    0.17%
      0     0    68826.0342   423    68924.8472     Cuts: 255    10680    0.14%

Repeating presolve.
Tried aggregator 2 times.
MIP Presolve eliminated 1332 rows and 53092 columns.
MIP Presolve modified 1257 coefficients.
Aggregator did 1212 substitutions.
Reduced MIP has 6596 rows, 79339 columns, and 381532 nonzeros.
Reduced MIP has 79280 binaries, 0 generals, 0 SOSs, and 0 indicators.
Preso

   1589  1196    68901.2045   675    68924.8472    68883.1636   210934    0.06%
   1604  1205    68901.8913   667    68924.8472    68883.1636   211533    0.06%
   1618  1194    68892.1503   733    68924.8472    68883.6873   216714    0.06%
   1639  1253    68915.1291   331    68924.8472    68883.6873   222067    0.06%
   1684  1223    68904.9050   594    68924.8472    68883.6873   218578    0.06%
Elapsed time = 224.58 sec. (212220.94 ticks, tree = 98.81 MB, solutions = 1)
   1723  1357    68919.1929   255    68924.8472    68883.6873   234626    0.06%
   1756  1360    68897.7619   701    68924.8472    68883.6873   236604    0.06%
   1813  1387        cutoff          68924.8472    68883.6873   236709    0.06%
   1842  1396    68903.3963   645    68924.8472    68883.6873   240183    0.06%
   1865  1420    68905.4269   674    68924.8472    68883.6873   243061    0.06%
   1891  1429    68906.8218   673    68924.8472    68883.6873   243755    0.06%
   1924  1411    68911.9119   603    68924.

KeyboardInterrupt: 

In [18]:
best_score - initial_score

0.0

## Final output

In [19]:
## Output

is_improved = new_score < initial_score
if is_valid and (is_improved or (path_dump_improved != path_init_conf)):
    print('output to', path_dump_improved)
    dump_conf(assigned_day_new, path_dump_improved)

output to ../output/m24-improved-random.csv


## Debug

In [20]:
# [
#     [assignment_matrix[family, c].solution_value() for c in range(N_choices)]
#     for family in range(10)
# ]        

In [21]:
# if N_choices >= N_choices_ori:
#     print([
#         [unpreferred_day_counts[day, size].solution_value() for size in possible_family_sizes]
#         for day in range(1, 10)
#     ])